In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import os

secret = os.environ.get("SPARK_AUTH_SECRET")
conf = SparkConf()
conf.set("spark.authenticate", "true")
conf.set("spark.driver.extraJavaOptions", f"-Dspark.authenticate.secret={secret}")

spark = SparkSession.builder.config(conf=conf)\
    .appName("merge_silver") \
    .master("spark://node1.dw.felicity.net.bd:7077,node2.dw.felicity.net.bd:7077")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 04:17:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/11 04:17:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/11 04:17:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/11 04:17:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/06/11 04:17:11 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to spark-events/app-20250611041705-0039.inprogress. This is unsupported


In [4]:
spark.sql(f"""
INSERT INTO gold.fact_sales_enriched
SELECT
    f.order_item_id,
    f.order_id,
    
    c.customer_id,
    c.full_name,
    c.email,
    c.phone_number,
    c.location_id,
    
    l.country,
    l.state,
    l.city,
    l.postal_code,
    
    p.product_id,
    p.product_name,
    p.category,
    p.description,
    p.price,
    p.in_stock,
    
    f.order_date,
    f.quantity,
    f.unit_price,
    f.total_price,
    f.status,
    f.updated_at,
    
    c.created_at AS customer_created_at,
    c.load_timestamp AS customer_load_timestamp,
    p.created_at AS product_created_at,
    p.load_timestamp AS product_load_timestamp,
    l.load_timestamp AS location_load_timestamp,
    
    f.load_timestamp
FROM silver.fact_sales f
JOIN silver.dim_customer c ON f.customer_id = c.customer_id
JOIN silver.dim_location l ON c.location_id = l.location_id
JOIN silver.dim_product p ON f.product_id = p.product_id;

""");